# Statistical averages and error bars

When using stochastic methods such as Monte Carlo, it is very
important to be able to give correct estimates of the statistical variance of computed physical observables.

Let us suppose that a simulation yields the series of $N$ samples $\xi_1, \ldots, \xi_N$. Every $\xi_i$ in this series
can be seen as a random variable whose average value is the physical observable $\xi$ that we are trying to compute. The best estimate for $\xi$ is **always** the sample average

\begin{equation*}
  \xi \leftarrow \frac{1}{N} \sum_{i=1}^N \xi_i
\end{equation*}

One has to be more careful when it comes to estimating the
statistical error bar.

## Independent samples

The situation is easiest when the samples $\xi_1, \ldots, \xi_N$ are all independent. This is for example the case when *direct sampling* is used. In this case, the statistical error for the
mean value of $\xi$ can be obtained from the central limit theorem
and we have

\begin{equation*}
  \mathrm{error} = \frac{\sigma}{\sqrt{N}}
\end{equation*}

where $\sigma$ is the standard deviation of
every single sample and can be estimated directly
from the samples

\begin{equation*}
  \sigma^2 = \langle \xi_k^2 \rangle - \langle \xi_k \rangle^2
  \leftarrow \frac{1}{N} \sum_{i=1}^N \xi_i^2 -
  \left( \frac{1}{N} \sum_{i=1}^N \xi_i \right)^2
\end{equation*}

From a practical point of view, if your samples are stored in an array `samples`, then the estimates for the mean and variance of the physical observable can be obtained with
```python
mean = np.average(samples)
error = np.std(samples) / np.sqrt(samples.shape[0])
```

## Correlated samples

Unfortunately, most of the time it is difficult to use direct sampling and a Markov chain Monte Carlo must be implemented. In that case, the samples $\xi_1, \ldots, \xi_N$ are *not independent*, they are *correlated*. You can still compute the average of a physical observable from the sample average. But you have to be careful when estimating the error bar. There are 3 possibilities to find the error bar.

### Producing independent Markov chains

The simplest solution is to generate several completely independent
Markov chains (all of length $N$). The averages obtained for every chain are independent and we can use the formula above. What you will observe with correlated samples is that the error bar on the estimate from a given chain of length $N$ is larger than what it would be for a chain of independent samples

\begin{equation*}
  \mathrm{error} = \frac{\sigma}{\sqrt{N_\mathrm{eff}}}
  \quad \mathrm{with} \quad N_\mathrm{eff} = \frac{N}{\tau}
\end{equation*}

where $N_\mathrm{eff}$ can be seen as the number of effectively
independent samples. The autocorrelation time $\tau$ measures
how many steps it takes to reach a new independent sample.

### The bunching algorithm

It is not very convenient to have to run a simulation several times in order to find the error bar. The bunching method (see Smac p.60-62)  is a way to compute the standard deviation from a single series of samples. The idea is to start from the original series $\{\xi^{(\ell)}_i\}$ with $N$ samples and then construct a new series $\{ \xi^{(\ell+1)} \}$ of $N/2$ samples by taking averages of pairs of consecutive samples of the original series:

\begin{equation*}
    \xi^{(\ell+1)}_i = \frac{1}{2} \left( \xi^{(\ell)}_{2i-1} + \xi^{(\ell)}_{2i} \right)
\end{equation*}
  
We can estimate the standard deviation of this new series with the naive formula above and then continue and bin the series again. If the bunching has been done many times, we can expect that the samples eventually become independant and that the standard deviation estimate will become correct. In practice, you can plot the standard deviation as a function of the bunching level $\ell$ and see if it has a plateau where the estimate is faithful. Here is an example of a code that applies the bunching procedure starting
from a list `samples` (the number of elements should be a power of 2)
```python
n_levels = 16
error = np.zeros(n_levels)
for i in range(n_levels):
    error[i] = np.std(samples) / np.sqrt(samples.shape[0])
    samples = np.average(samples.reshape(-1, 2), axis=1)
```

In [7]:
import numpy as np
x=np.arange(8)
x=x.reshape(-1,2)
x

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

### Using the autocorrelation function

The degree of correlations between samples can be analyzed using the covariance matrix

\begin{equation*}
  C_{ij} = \left \langle (\xi_i -\langle \xi_i \rangle) \, (\xi_j -\langle \xi_j \rangle) \right \rangle
\end{equation*}

After the transient, the covariance is just a function $C(n)$ of the
distance $n = |i-j|$. Clearly, $C(n)$ is vanishing if the samples are independent. When they are correlated $C(n)$ typically decays exponentially with $n$

\begin{equation*}
  C(n) \simeq \sigma^2 \exp(-2 n / \tau)
\end{equation*}

Again, the autocorrelation time $\tau$ appears as it controls
how quickly samples become independent. It can be obtained from

\begin{equation*}
  \tau = \frac{1}{\sigma^2} \left( C(0) + 2 \sum_{n=1}^N C(n) \right)
\end{equation*}

The error is then simply

\begin{equation*}
  \mathrm{error} = \frac{\sigma \sqrt{\tau}}{\sqrt{N}}
  = \sqrt{\frac{C(0) + 2 \sum_{n=1}^N C(n)}{N}}
\end{equation*}